In [0]:
pip install reverse_geocoder

In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey="Bronze", key = "bronze_output")

start_date = bronze_output.get("start_date", "")
silver_s3 = bronze_output.get("silver_s3", "")
gold_s3 = bronze_output.get("gold_s3", "")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta
import reverse_geocoder as rg

In [0]:
df = spark.read.parquet(silver_s3).filter(col('time') > start_date)

In [0]:
def get_country_code(lat, lon):
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        return result
    except Exception as e:
        print(f"Error Processing Coodinates: {lat}, {lon} -> str(e)")
        return None

In [0]:
get_country_code_udf = udf(get_country_code, StringType())

In [0]:
dfWithLocation = df.withColumn('country_code', get_country_code_udf(col('latitude'), col('longitude')))

In [0]:
dfWithLocation.write.mode("append").parquet(gold_s3)